# Assignment 5

# Part 1

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('homework/radar_parameters.csv')

x_data = df.iloc[:,1:7]
y_data = df.iloc[:,7]

In [3]:
x_data

,Zh (dBZ),Zdr (dB),Ldr (dB),Kdp (deg km-1),Ah (dBZ/km),Adr (dB/km)
0,23.144878,0.418637,-41.757733,0.005395,0.000290,0.000012
1,22.737156,0.322850,-43.772069,0.005194,0.000360,0.000012
2,26.869826,0.330948,-43.577399,0.013385,0.000903,0.000030
3,28.540561,0.399480,-42.139731,0.018872,0.001036,0.000043
4,30.500127,0.543758,-39.763087,0.027438,0.001157,0.000064
...,...,...,...,...,...,...
18964,31.515997,0.579955,-39.244229,0.034048,0.001417,0.000080
18965,29.993334,0.567935,-39.399188,0.024134,0.001032,0.000057
18966,31.685913,0.655681,-38.375696,0.033971,0.001165,0.000081
18967,32.980096,0.768586,-37.166218,0.043117,0.001285,0.000105


In [4]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.3)

In [5]:
x_train

,Zh (dBZ),Zdr (dB),Ldr (dB),Kdp (deg km-1),Ah (dBZ/km),Adr (dB/km)
2917,25.305783,0.420198,-41.708667,0.008872,0.000489,0.000020
11443,19.094886,0.376209,-42.556861,0.002176,0.000134,0.000005
17111,39.405696,1.329290,-33.059984,0.139483,0.002509,0.000381
3104,27.902502,0.498867,-40.380558,0.015472,0.000754,0.000036
61,31.055886,0.432701,-41.498536,0.033097,0.001737,0.000076
...,...,...,...,...,...,...
2945,28.098705,0.411829,-41.898594,0.016932,0.000909,0.000039
7835,30.912519,0.704130,-37.813452,0.027723,0.000911,0.000066
11364,40.587882,1.023231,-35.033344,0.216509,0.004831,0.000552
16798,34.146045,0.845016,-36.439626,0.054136,0.001493,0.000133


In [6]:
x_test

,Zh (dBZ),Zdr (dB),Ldr (dB),Kdp (deg km-1),Ah (dBZ/km),Adr (dB/km)
11938,31.304453,0.811978,-36.729440,0.028647,0.000831,0.000070
1810,33.053537,0.829969,-36.603550,0.042428,0.001156,0.000104
16532,24.701658,0.591120,-39.131607,0.007042,0.000274,0.000017
9331,27.099436,0.373788,-42.595956,0.013769,0.000871,0.000031
16519,24.815387,0.441083,-41.327298,0.007838,0.000420,0.000018
...,...,...,...,...,...,...
15676,37.707656,0.947035,-35.607471,0.116295,0.002806,0.000292
5107,25.744253,0.626881,-38.686784,0.008786,0.000324,0.000021
14005,30.930593,0.474990,-40.801970,0.031408,0.001484,0.000073
6586,28.615795,0.625422,-38.683020,0.017041,0.000649,0.000040


# Part 2

In [7]:
from sklearn.linear_model import LinearRegression
model_linear = LinearRegression(fit_intercept=True)

model_linear.fit(x_train, y_train)

LinearRegression()

In [8]:
y_test_predicted = model_linear.predict(x_test)

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

rmse_linear = sqrt(mean_squared_error(y_test, y_test_predicted))
r2_linear = r2_score(y_test, y_test_predicted)

In [9]:
df_z = 10**(x_data['Zh (dBZ)']/10)
df_r = (df_z/200)**(1/1.6)

baseline_rms = sqrt(mean_squared_error(y_data, df_r))
baseline_r2 = r2_score(y_data, df_r)

In [10]:
print('RMSE of MLR:', rmse_linear)
print('R^2 of MLR:', r2_linear)

print('RMSE of baseline:', baseline_rms)
print('R^2 of baseline:', baseline_r2)

RMSE of MLR: 0.9420616042289477
R^2 of MLR: 0.988487617518314
RMSE of baseline: 7.157590840042378
R^2 of baseline: 0.3023229070437503


# Part 3

In [11]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree),
                         LinearRegression(**kwargs))

In [12]:
from sklearn.model_selection import GridSearchCV

param_grid = {'polynomialfeatures__degree': np.arange(7)}

poly_grid = GridSearchCV(PolynomialRegression(), param_grid, scoring= 'r2', cv=7)

In [13]:
poly_grid.fit(x_train, y_train)

GridSearchCV(cv=7,
             estimator=Pipeline(steps=[('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('linearregression',
                                        LinearRegression())]),
             param_grid={'polynomialfeatures__degree': array([0, 1, 2, 3, 4, 5, 6])},
             scoring='r2')

In [14]:
poly_grid.best_params_

{'polynomialfeatures__degree': 2}

In [15]:
poly_model = poly_grid.best_estimator_

In [16]:
y_test_poly_pred = poly_model.fit(x_train, y_train).predict(x_test)

In [17]:
rmse_poly = sqrt(mean_squared_error(y_test, y_test_poly_pred))
r2_poly = r2_score(y_test, y_test_poly_pred)

In [18]:
print('RMSE of grid:', rmse_poly)
print('R^2 of grid:', r2_poly)

print('RMSE of MLR:', rmse_linear)
print('R^2 of MLR:', r2_linear)

print('RMSE of baseline:', baseline_rms)
print('R^2 of baseline:', baseline_r2)

RMSE of grid: 0.21769848653700608
R^2 of grid: 0.9993852232673677
RMSE of MLR: 0.9420616042289477
R^2 of MLR: 0.988487617518314
RMSE of baseline: 7.157590840042378
R^2 of baseline: 0.3023229070437503


# Part 4

In [19]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(50)

In [20]:
random_param_grid = {'bootstrap': [True, False],  
                        'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],  
                        'max_features': ['auto', 'sqrt'],  
                        'min_samples_leaf': [1, 2, 4],  
                        'min_samples_split': [2, 5, 10],  
                        'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [21]:
from sklearn.model_selection import RandomizedSearchCV

forest_grid = RandomizedSearchCV(estimator = forest, param_distributions= random_param_grid, scoring= 'r2')

In [23]:
forest_grid.fit(x_train, y_train)

c:\Users\Charu\miniconda3\envs\py.3\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
c:\Users\Charu\miniconda3\envs\py.3\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
c:\Users\Charu\miniconda3\envs\py.3\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also th

RandomizedSearchCV(estimator=RandomForestRegressor(n_estimators=50),
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   scoring='r2')

In [24]:
forest_grid.best_params_

{'n_estimators': 1200,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': None,
 'bootstrap': False}

In [25]:
forest_model = forest_grid.best_estimator_

In [27]:
y_test_forest_pred = forest_model.fit(x_train, y_train).predict(x_test)

In [28]:
rmse_forest = sqrt(mean_squared_error(y_test, y_test_forest_pred))
r2_forest = r2_score(y_test, y_test_forest_pred)

In [29]:
print('RMSE of random forest:', rmse_forest)
print('R^2 of random forest:', r2_forest)

print('RMSE of grid:', rmse_poly)
print('R^2 of grid:', r2_poly)

print('RMSE of MLR:', rmse_linear)
print('R^2 of MLR:', r2_linear)

print('RMSE of baseline:', baseline_rms)
print('R^2 of baseline:', baseline_r2)

RMSE of random forest: 1.2693003462040438
R^2 of random forest: 0.9791005238823449
RMSE of grid: 0.21769848653700608
R^2 of grid: 0.9993852232673677
RMSE of MLR: 0.9420616042289477
R^2 of MLR: 0.988487617518314
RMSE of baseline: 7.157590840042378
R^2 of baseline: 0.3023229070437503
